In [ ]:
import requests
import time
import json
import pandas as pd
from rapidfuzz import fuzz, process
import re

# ---------------------------
# USDA API CONFIG
# ---------------------------

load_dotenv()                                     
api_key = os.getenv("USDA_API_KEY")               

if api_key is None:
    raise ValueError("USDA_API_KEY not found in .env file")
base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"

# ---------------------------
# Food-101 labels (101 items)
# ---------------------------
food_101_labels = sorted(set([
    "apple_pie","baby_back_ribs","baklava","beef_carpaccio","beef_tartare",
    "beet_salad","beignets","bibimbap","bread_pudding","breakfast_burrito",
    "bruschetta","caesar_salad","cannoli","caprese_salad","carrot_cake",
    "ceviche","cheesecake","cheese_plate","chicken_curry","chicken_quesadilla",
    "chicken_wings","chocolate_cake","chocolate_mousse","churros","clam_chowder",
    "club_sandwich","crab_cakes","creme_brulee","croque_madame","cup_cakes",
    "deviled_eggs","donuts","dumplings","edamame","eggs_benedict","escargots",
    "falafel","filet_mignon","fish_and_chips","foie_gras","french_fries",
    "french_onion_soup","french_toast","fried_calamari","fried_rice","frozen_yogurt",
    "garlic_bread","gnocchi","greek_salad","grilled_cheese_sandwich","grilled_salmon",
    "guacamole","gyoza","hamburger","hot_and_sour_soup","hot_dog","huevos_rancheros",
    "hummus","ice_cream","lasagna","lobster_bisque","lobster_roll_sandwich","macaroni_and_cheese",
    "macarons","miso_soup","mussels","nachos","omelette","onion_rings","oysters",
    "pad_thai","paella","pancakes","panna_cotta","peking_duck","pho","pizza",
    "pork_chop","poutine","prime_rib","pulled_pork_sandwich","ramen","ravioli",
    "red_velvet_cake","risotto","samosa","sashimi","scallops","seaweed_salad",
    "shrimp_and_grits","spaghetti_bolognese","spaghetti_carbonara","spring_rolls",
    "steak","strawberry_shortcake","sushi","tacos","takoyaki","tiramisu","tuna_tartare","waffles"
]))

len(food_101_labels)

101

In [6]:
# Cell 2: Synonyms and alternate search terms

synonyms = {
    "beef_carpaccio": "beef carpaccio",
    "beef_tartare": "steak tartare",
    "bibimbap": "korean rice bowl",
    "club_sandwich": "club sandwich",
    "croque_madame": "croque madame sandwich",
    "fish_and_chips": "fish and chips",
    "foie_gras": "duck liver pate",
    "gyoza": "gyoza dumplings",
    "lobster_roll_sandwich": "lobster roll",
    "omelette": "egg omelet",
    "oysters": "raw oysters",
    "pho": "vietnamese pho soup",
    "pork_chop": "grilled pork chop",
    "poutine": "poutine fries",
    "prime_rib": "prime rib roast",
    "spaghetti_bolognese": "spaghetti with meat sauce",
    "spring_rolls": "spring rolls",
    "sushi": "sushi rolls",
    "tacos": "mexican tacos",
    "takoyaki": "takoyaki balls",
    "tuna_tartare": "tuna tartare",
    "ramen": "japanese ramen noodle soup",
    "paella": "spanish paella",
    "samosa": "indian samosa",
    "ceviche": "fish ceviche",
    "panna_cotta": "panna cotta dessert",
    "beignets": "french beignets",
    "bruschetta": "bruschetta appetizer",
    "guacamole": "guacamole avocado dip",
    "falafel": "falafel balls",
    "lasagna": "lasagna with meat",
    "fried_rice": "fried rice"
}

# Extra, more descriptive variants
extra_synonyms = {
    "beef_carpaccio": "thinly sliced raw beef appetizer",
    "bibimbap": "korean mixed rice bowl with vegetables and egg",
    "club_sandwich": "layered chicken bacon sandwich",
    "croque_madame": "grilled ham and cheese sandwich with fried egg",
    "fish_and_chips": "battered fried fish served with potato fries",
    "foie_gras": "goose liver pate french delicacy",
    "omelette": "fluffy egg omelet with cheese",
    "oysters": "fresh raw oysters shellfish",
    "pho": "beef noodle soup vietnamese",
    "pork_chop": "fried pork meat with bone",
    "poutine": "canadian french fries with cheese curds and gravy",
    "prime_rib": "slow roasted beef rib cut",
    "spring_rolls": "crispy fried vegetable rolls",
    "sushi": "japanese rice rolls with fish",
    "tacos": "mexican tortilla with meat and vegetables",
    "takoyaki": "japanese octopus dough balls",
    "tuna_tartare": "raw minced tuna appetizer",
    "ramen": "japanese noodle soup",
    "paella": "spanish seafood rice dish",
    "samosa": "indian fried pastry with potato filling",
    "ceviche": "raw fish marinated in lemon juice",
    "panna_cotta": "italian creamy milk dessert",
    "beignets": "fried dough pastry french",
    "bruschetta": "grilled bread with tomato topping",
    "guacamole": "avocado dip",
    "falafel": "fried chickpea balls",
    "lasagna": "layered pasta with cheese and tomato sauce",
    "fried_rice": "stir fried rice with egg and vegetables"
}

# Merge extra_synonyms into synonyms
for k, v in extra_synonyms.items():
    # keep both versions as separate search terms later
    if k in synonyms:
        synonyms[k] = synonyms[k] + " || " + v
    else:
        synonyms[k] = v

synonyms

{'beef_carpaccio': 'beef carpaccio || thinly sliced raw beef appetizer',
 'beef_tartare': 'steak tartare',
 'bibimbap': 'korean rice bowl || korean mixed rice bowl with vegetables and egg',
 'club_sandwich': 'club sandwich || layered chicken bacon sandwich',
 'croque_madame': 'croque madame sandwich || grilled ham and cheese sandwich with fried egg',
 'fish_and_chips': 'fish and chips || battered fried fish served with potato fries',
 'foie_gras': 'duck liver pate || goose liver pate french delicacy',
 'gyoza': 'gyoza dumplings',
 'lobster_roll_sandwich': 'lobster roll',
 'omelette': 'egg omelet || fluffy egg omelet with cheese',
 'oysters': 'raw oysters || fresh raw oysters shellfish',
 'pho': 'vietnamese pho soup || beef noodle soup vietnamese',
 'pork_chop': 'grilled pork chop || fried pork meat with bone',
 'poutine': 'poutine fries || canadian french fries with cheese curds and gravy',
 'prime_rib': 'prime rib roast || slow roasted beef rib cut',
 'spaghetti_bolognese': 'spaghetti

In [7]:
# Cell 3: Helper functions

def normalize_text(text: str) -> str:
    """Lowercase, keep letters/spaces, collapse multiple spaces."""
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def usda_search(query: str, page_size: int = 10):
    """Call USDA search API for a given query string."""
    params = {
        "query": query,
        "pageSize": page_size,
        "api_key": api_key
    }
    try:
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()
        data = resp.json()
        return data.get("foods", [])
    except Exception as e:
        print(f"[ERROR] USDA API call failed for query='{query}': {e}")
        return []

def extract_nutrients(food_obj: dict):
    """Extract calories, protein, fat, carbs from a USDA food object."""
    nutrients_list = food_obj.get("foodNutrients", [])
    nutrients = {n.get("nutrientName"): n.get("value") for n in nutrients_list}

    calories = nutrients.get("Energy", None)
    protein = nutrients.get("Protein", None)
    fat = nutrients.get("Total lipid (fat)", None)
    carbs = nutrients.get("Carbohydrate, by difference", None)

    return calories, protein, fat, carbs

In [8]:
# Cell 4: Build search terms per label

label_to_search_terms = {}

for label in food_101_labels:
    base = label.replace("_", " ")
    terms = [base]

    # If we have synonyms, split the combined string into multiple terms
    if label in synonyms:
        syn_str = synonyms[label]
        for part in syn_str.split("||"):
            t = part.strip()
            if t:
                terms.append(t)

    # Remove duplicates while preserving order
    seen = set()
    uniq_terms = []
    for t in terms:
        if t not in seen:
            uniq_terms.append(t)
            seen.add(t)

    label_to_search_terms[label] = uniq_terms

# Quick sanity check for a few labels
for k in ["beef_carpaccio", "samosa", "tuna_tartare", "tacos"]:
    print(k, "->", label_to_search_terms.get(k))

beef_carpaccio -> ['beef carpaccio', 'thinly sliced raw beef appetizer']
samosa -> ['samosa', 'indian samosa', 'indian fried pastry with potato filling']
tuna_tartare -> ['tuna tartare', 'raw minced tuna appetizer']
tacos -> ['tacos', 'mexican tacos', 'mexican tortilla with meat and vegetables']


In [9]:
# Cell 5: Download raw candidates for all Food-101 labels

all_rows = []

for label in food_101_labels:
    terms = label_to_search_terms[label]
    print(f"\n==== {label} ====")
    for q in terms:
        q_stripped = q.strip()
        if not q_stripped:
            continue

        print(f"  Querying USDA for: '{q_stripped}'")
        foods = usda_search(q_stripped, page_size=15)

        for f in foods:
            calories, protein, fat, carbs = extract_nutrients(f)
            row = {
                "source_label": label,
                "query_used": q_stripped,
                "fdcId": f.get("fdcId"),
                "description": f.get("description"),
                "dataType": f.get("dataType"),
                "calories": calories,
                "protein": protein,
                "fat": fat,
                "carbohydrates": carbs
            }
            all_rows.append(row)

        # be nice to the API
        time.sleep(0.25)

len(all_rows)


==== apple_pie ====
  Querying USDA for: 'apple pie'

==== baby_back_ribs ====
  Querying USDA for: 'baby back ribs'

==== baklava ====
  Querying USDA for: 'baklava'

==== beef_carpaccio ====
  Querying USDA for: 'beef carpaccio'
  Querying USDA for: 'thinly sliced raw beef appetizer'

==== beef_tartare ====
  Querying USDA for: 'beef tartare'
  Querying USDA for: 'steak tartare'

==== beet_salad ====
  Querying USDA for: 'beet salad'

==== beignets ====
  Querying USDA for: 'beignets'
  Querying USDA for: 'french beignets'
  Querying USDA for: 'fried dough pastry french'

==== bibimbap ====
  Querying USDA for: 'bibimbap'
  Querying USDA for: 'korean rice bowl'
  Querying USDA for: 'korean mixed rice bowl with vegetables and egg'

==== bread_pudding ====
  Querying USDA for: 'bread pudding'

==== breakfast_burrito ====
  Querying USDA for: 'breakfast burrito'

==== bruschetta ====
  Querying USDA for: 'bruschetta'
  Querying USDA for: 'bruschetta appetizer'
  Querying USDA for: 'gri


==== tuna_tartare ====
  Querying USDA for: 'tuna tartare'
  Querying USDA for: 'raw minced tuna appetizer'

==== waffles ====
  Querying USDA for: 'waffles'


2264

In [10]:
# Cell 6: Save raw candidate table & quick inspection

raw_df = pd.DataFrame(all_rows)

print("Total candidate rows:", len(raw_df))
print("Unique Food-101 labels in candidates:", raw_df["source_label"].nunique())

# Normalize description for later fuzzy matching
raw_df["normalized_description"] = raw_df["description"].astype(str).apply(normalize_text)

# Save raw candidates for debugging / backup
raw_df.to_csv("../data/raw/usda_food_candidates_raw.csv", index=False)

raw_df.head()

Total candidate rows: 2264
Unique Food-101 labels in candidates: 101


,source_label,query_used,fdcId,description,dataType,calories,protein,fat,carbohydrates,normalized_description
0,apple_pie,apple pie,2084465,APPLE PIE,Branded,300.0,33.30,8.33,41.7,apple pie
1,apple_pie,apple pie,1916963,APPLE PIE,Branded,276.0,3.25,8.13,46.3,apple pie
2,apple_pie,apple pie,2288447,APPLE PIE,Branded,354.0,1.77,15.00,54.0,apple pie
3,apple_pie,apple pie,2063662,APPLE PIE,Branded,211.0,1.32,11.00,29.1,apple pie
4,apple_pie,apple pie,2063720,APPLE PIE,Branded,254.0,2.31,12.30,33.1,apple pie


In [11]:
# Cell 7: Fuzzy matching to select the best row per Food-101 label

final_rows = []
all_norm_descriptions = raw_df["normalized_description"].tolist()

for label in food_101_labels:
    candidates = raw_df[raw_df["source_label"] == label].copy()

    # Build target texts for scoring (base label + all search terms)
    base = label.replace("_", " ")
    terms = label_to_search_terms[label]
    targets = [normalize_text(t) for t in ([base] + terms)]

    def score_candidate(desc_norm: str) -> int:
        # Score = max token_sort_ratio against any target
        return max(fuzz.token_sort_ratio(desc_norm, t) for t in targets)

    if not candidates.empty:
        # Compute scores within candidates of this label
        candidates["match_score"] = candidates["normalized_description"].apply(score_candidate)

        best_row = candidates.sort_values("match_score", ascending=False).iloc[0]
        final_rows.append(best_row.to_dict())
        print(f"[OK] Label '{label}' matched with '{best_row['description']}' (score={best_row['match_score']})")
    else:
        # Fallback: search globally in all rows
        print(f"[WARN] No direct candidates for '{label}'. Using global fuzzy search.")
        # Build one combined query string from terms
        combined_query = base
        # use process.extractOne on all normalized descriptions
        match = process.extractOne(combined_query, all_norm_descriptions, scorer=fuzz.token_sort_ratio)
        if match:
            best_desc_norm = match[0]
            row = raw_df[raw_df["normalized_description"] == best_desc_norm].iloc[0]
            row = row.copy()
            row["match_score"] = match[1]
            # override source_label to correct label
            row["source_label"] = label
            final_rows.append(row.to_dict())
            print(f"   -> matched to '{row['description']}' (score={row['match_score']})")
        else:
            print(f"[ERROR] Could not match ANY USDA row for label '{label}'")

len(final_rows)

[OK] Label 'apple_pie' matched with 'APPLE PIE' (score=100.0)
[OK] Label 'baby_back_ribs' matched with 'BABY BACK RIBS' (score=100.0)
[OK] Label 'baklava' matched with 'Baklava' (score=100.0)
[OK] Label 'beef_carpaccio' matched with 'THINLY SLICED BEEF' (score=72.0)
[OK] Label 'beef_tartare' matched with 'Steak tartare' (score=100.0)
[OK] Label 'beet_salad' matched with 'BEET SALAD' (score=100.0)
[OK] Label 'beignets' matched with 'Beignet' (score=93.33333333333333)
[OK] Label 'bibimbap' matched with 'Bibimbap, Korean' (score=69.56521739130434)
[OK] Label 'bread_pudding' matched with 'BREAD PUDDING' (score=100.0)
[OK] Label 'breakfast_burrito' matched with 'BREAKFAST BURRITO' (score=100.0)
[OK] Label 'bruschetta' matched with 'Bruschetta' (score=100.0)
[OK] Label 'caesar_salad' matched with 'CAESAR SALAD' (score=100.0)
[OK] Label 'cannoli' matched with 'CANNOLI' (score=100.0)
[OK] Label 'caprese_salad' matched with 'CAPRESE SALAD' (score=100.0)
[OK] Label 'carrot_cake' matched with 'CA

101

In [12]:
# Cell 8: Build and save final USDA dataset (one row per Food-101 label)

final_df = pd.DataFrame(final_rows)

# For clarity, create a column 'query' that is just the human-readable food name
final_df["query"] = final_df["source_label"].apply(lambda x: x.replace("_", " "))

# Reorder columns (you can adjust as you like)
final_df = final_df[[
    "query",
    "fdcId",
    "description",
    "dataType",
    "calories",
    "protein",
    "fat",
    "carbohydrates",
    "source_label",
    "query_used",
    "match_score"
]]

print("Final rows:", len(final_df))
print("Unique labels:", final_df["source_label"].nunique())

# Save as the main CSV your backend & notebooks will use
final_df.to_csv("../data/raw/usda_food_data.csv", index=False)

final_df.head(10)

Final rows: 101
Unique labels: 101


,query,fdcId,description,dataType,calories,protein,fat,carbohydrates,source_label,query_used,match_score
0,apple pie,2084465,APPLE PIE,Branded,300.0,33.30,8.33,41.70,apple_pie,apple pie,100.000000
1,baby back ribs,1457876,BABY BACK RIBS,Branded,170.0,18.80,9.82,0.00,baby_back_ribs,baby back ribs,100.000000
2,baklava,2708044,Baklava,Survey (FNDDS),440.0,6.58,29.34,37.55,baklava,baklava,100.000000
3,beef carpaccio,2660667,THINLY SLICED BEEF,Branded,179.0,14.30,12.90,1.43,beef_carpaccio,thinly sliced raw beef appetizer,72.000000
4,beef tartare,2706394,Steak tartare,Survey (FNDDS),216.0,17.45,15.60,0.34,beef_tartare,beef tartare,100.000000
5,beet salad,1922885,BEET SALAD,Branded,93.0,1.32,5.29,11.00,beet_salad,beet salad,100.000000
6,beignets,2708071,Beignet,Survey (FNDDS),417.0,5.46,20.18,53.69,beignets,beignets,93.333333
7,bibimbap,2708950,"Bibimbap, Korean",Survey (FNDDS),76.0,5.50,2.44,8.11,bibimbap,bibimbap,69.565217
8,bread pudding,2381798,BREAD PUDDING,Branded,222.0,4.04,4.55,42.40,bread_pudding,bread pudding,100.000000
9,breakfast burrito,2315891,BREAKFAST BURRITO,Branded,198.0,8.13,7.77,26.20,breakfast_burrito,breakfast burrito,100.000000


In [13]:
# Cell 9: Verification that all 101 labels are covered

present_labels = set(final_df["source_label"].tolist())
missing = [lbl for lbl in food_101_labels if lbl not in present_labels]

print("Missing labels:", missing)
print("Count missing:", len(missing))

Missing labels: []
Count missing: 0
